In [4]:
# load tydi json and compute stats on the questions and answers
import glob
import gzip
import json
import spacy

nlp = spacy.load('en_core_web_sm')

train_data_dir = "/dccstor/srosent2/primeqa/data/train/nq-lfqa/nq-train*jsonl"
dev_data_dir = "/dccstor/srosent2/primeqa/data/dev/nq-lfqa/*jsonl"

train_files = glob.glob(train_data_dir)
dev_files = glob.glob(dev_data_dir)

In [2]:

def load_json_from_file(gt_file_patterns):
    data = []
    if gt_file_patterns.endswith('gz'):
        f = gzip.open(gt_file_patterns, 'rt', encoding='utf-8')
    else:
        f = open(gt_file_patterns, 'rt', encoding='utf-8')
    lines = f.readlines()
    for line in lines:
        data.append(json.loads(line))
    return data


In [ ]:
from random import sample

def compute_stats(data, type):
    first_word_stat = {}
    for example in data:
        qtype = example['type'][0]
        
        if qtype != type:
            continue
        example_id = example['example_id']
        question = example["question_text"]

        q_word = question.split()[0]

        annotation = example['annotations'][0]

        if annotation == None:
            continue

        minimal_text = ""
        if annotation['minimal_answer']['plaintext_start_byte'] != -1:
            minimal_text = example['document_plaintext'].encode('utf-8')[annotation['minimal_answer']['plaintext_start_byte']:annotation['minimal_answer']['plaintext_end_byte']].decode('utf-8')
        passage_offsets = example['passage_answer_candidates'][annotation['passage_answer']['candidate_index']]
        passage_text = example['document_plaintext'].encode('utf-8')[passage_offsets['plaintext_start_byte']:passage_offsets['plaintext_end_byte']].decode('utf-8')
        info = {}
        info["question"] = question
        info["passage_text"] = passage_text
        info["minimal_text"] = minimal_text
        info["example_id"] = example_id
        if q_word in first_word_stat:
            first_word_stat[q_word].append(info)
        else:
            first_word_stat[q_word] = []
            first_word_stat[q_word].append(info)
    for word in first_word_stat:
        if len(first_word_stat[word]) > 20:
            sample_questions = sample(first_word_stat[word],5)
            print(word + ": " + str(len(first_word_stat[word])))
            for sample_question in sample_questions:
                print(str(sample_question['example_id']) + "\t" + sample_question['question'] + "\t" + sample_question['passage_text'] + "\t" + sample_question['minimal_text'])
                    

In [ ]:
dev_data = []
for dev_file in dev_files:
    dev_data.extend(load_json_from_file(dev_file))

compute_stats(dev_data, "la")

In [6]:
train_data = []
for train_file in train_files:
    train_data.extend(load_json_from_file(train_file))

In [ ]:
compute_stats(train_data, "la")

In [15]:
def get_appen_data(data, type):
    count = 0
    for example in data:
        qtype = example['type'][0]
        
        if qtype != type:
            continue
        example_id = example['example_id']
        question = example["question_text"]

        q_word = question.split()[0]

        annotation = example['annotations'][0]

        if annotation == None:
            continue

        minimal_text = ""
        if annotation['minimal_answer']['plaintext_start_byte'] != -1:
            minimal_text = example['document_plaintext'].encode('utf-8')[annotation['minimal_answer']['plaintext_start_byte']:annotation['minimal_answer']['plaintext_end_byte']].decode('utf-8')
        passage_offsets = example['passage_answer_candidates'][annotation['passage_answer']['candidate_index']]
        passage_text = example['document_plaintext'].encode('utf-8')[passage_offsets['plaintext_start_byte']:passage_offsets['plaintext_end_byte']].decode('utf-8')
        
        passage_sentences = ""
        for sentence in nlp(passage_text).sents:
            passage_sentences += sentence.text + "&nbsp;"

        print(str(example_id) + "\t" + question + "\t" + passage_sentences + "\t" + minimal_text)
        count += 1
        if count > 200:
            break
             

In [16]:
get_appen_data(train_data, "la")

-2664875860902576660	what is the purpose of the united states criminal justice system	In the United States , criminal justice policy has been guided by the 1967 President 's Commission on Law Enforcement and Administration of Justice , which issued a ground - breaking report `` The Challenge of Crime in a Free Society '' .&nbsp;This report made more than 200 recommendations as part of a comprehensive approach toward the prevention and fighting of crime .&nbsp;Some of those recommendations found their way into the Omnibus Crime Control and Safe Streets Act of 1968 .&nbsp;The Commission advocated a `` systems '' approach to criminal justice , with improved coordination among law enforcement , courts , and correctional agencies .&nbsp;The President 's Commission defined the criminal justice system as the means for society to `` enforce the standards of conduct necessary to protect individuals and the community . ''&nbsp;	the means for society to `` enforce the standards of conduct necessa